In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_2025 = pd.read_csv('/home/pedro/Projetos/samu -  banco de dados/SAMU-Equipe-4/dados/Dados samu 2025.csv')

df_2025.head(10)

,_id,2025-01-01,00:00:45,RECIFE,BOA VISTA,AV DA BOA VISTA,RESIDENCIAL,DROGAS,ALCOOLISMO,MASCULINO,25,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
0,1,2025-01-01T00:00:00,00:08:03,POMBOS,ALTO DO FRADE,R SARDINHA,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO ENVOLVENDO MOTO,MASCULINO,22.0,NaN,1. OCORRÊNCIA CONCLUÍDA COM ÊXITO
1,2,2025-01-01T00:00:00,00:08:35,PAULISTA,NOSSA SENHORA DO O,AV CLAUDIO JOSE GUEIROS LEIT NOSSA.SENHORA.DO.O,RESIDENCIAL,CAUSAS EXTERNAS,QUEDA DA PROPRIA ALTURA,FEMININO,45.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
2,3,2025-01-01T00:00:00,00:17:44,JABOATAO DOS GUARARAPES,MARCOS FREIRE,RUA DOMINGO FERNANDES N,RESIDENCIAL,DROGAS,INTOXICACAO EXOGENA,FEMININO,46.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO
3,4,2025-01-01T00:00:00,00:19:24,IGARASSU,CRUZ DE REBOUCAS,R JOCA RODRIGUES,VIA PÚBLICA,GASTROINTESTINAL,DOR ABDOMINAL,MASCULINO,65.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO
4,5,2025-01-01T00:00:00,00:20:55,ESCADA,ESCADA CENTRO,BR,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO COM CARROS,MASCULINO,57.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO
5,6,2025-01-01T00:00:00,00:21:11,JABOATAO DOS GUARARAPES,PIEDADE,TRAV TERCEIRA DA RUA SUCUPIRA DO NORTE,VIA PÚBLICA,NEUROLOGICA,CONFUSAO MENTAL,MASCULINO,56.0,NaN,1. OCORRÊNCIA CONCLUÍDA COM ÊXITO
6,7,2025-01-01T00:00:00,00:22:49,SIRINHAEM,BARRA DE SIRINHAEM (DISTRITO),COND TOQUINHO,VIA PÚBLICA,GERAIS/OUTROS,OUTROS,MASCULINO,NaN,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
7,8,2025-01-01T00:00:00,00:22:49,SIRINHAEM,BARRA DE SIRINHAEM (DISTRITO),COND TOQUINHO,VIA PÚBLICA,GERAIS/OUTROS,OUTROS,MASCULINO,NaN,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
8,9,2025-01-01T00:00:00,00:26:51,RECIFE,CORDEIRO,ESTRADA FORTE DO ARRAIAL BOM JESUS,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO ENVOLVENDO MOTO,NaN,NaN,NaN,3. SOLICITAÇÃO DUPLICADA
9,10,2025-01-01T00:00:00,00:26:55,RECIFE,TORROES,AVENIDA ARRAIAL DO BOM JESUS,VIA PÚBLICA,CAUSAS EXTERNAS,OUTROS,MASCULINO,NaN,SOLICITAÇÃO DUPLICADA,SEM DESFECHO


In [3]:
nomes_colunas = [
    'ID', 'DATA', 'HORA_MINUTO', 'MUNICIPIO', 'BAIRRO',
    'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO', 'SEXO',
    'IDADE', 'MOTIVO_FINALIZACAO', 'MOTIVO_DESFECHO'
]

df_2025 = pd.read_csv('/home/pedro/Projetos/samu -  banco de dados/SAMU-Equipe-4/dados/Dados samu 2025.csv', header=None, names=nomes_colunas)

df_2025.head()

/tmp/ipykernel_21124/2994693486.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2025 = pd.read_csv('/home/pedro/Projetos/samu -  banco de dados/SAMU-Equipe-4/dados/Dados samu 2025.csv', header=None, names=nomes_colunas)


,ID,DATA,HORA_MINUTO,MUNICIPIO,BAIRRO,ENDERECO,ORIGEM_CHAMADO,TIPO,SUBTIPO,SEXO,IDADE,MOTIVO_FINALIZACAO,MOTIVO_DESFECHO
0,_id,2025-01-01,00:00:45,RECIFE,BOA VISTA,AV DA BOA VISTA,RESIDENCIAL,DROGAS,ALCOOLISMO,MASCULINO,25.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
1,1,2025-01-01T00:00:00,00:08:03,POMBOS,ALTO DO FRADE,R SARDINHA,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO ENVOLVENDO MOTO,MASCULINO,22.0,NaN,1. OCORRÊNCIA CONCLUÍDA COM ÊXITO
2,2,2025-01-01T00:00:00,00:08:35,PAULISTA,NOSSA SENHORA DO O,AV CLAUDIO JOSE GUEIROS LEIT NOSSA.SENHORA.DO.O,RESIDENCIAL,CAUSAS EXTERNAS,QUEDA DA PROPRIA ALTURA,FEMININO,45.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
3,3,2025-01-01T00:00:00,00:17:44,JABOATAO DOS GUARARAPES,MARCOS FREIRE,RUA DOMINGO FERNANDES N,RESIDENCIAL,DROGAS,INTOXICACAO EXOGENA,FEMININO,46.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO
4,4,2025-01-01T00:00:00,00:19:24,IGARASSU,CRUZ DE REBOUCAS,R JOCA RODRIGUES,VIA PÚBLICA,GASTROINTESTINAL,DOR ABDOMINAL,MASCULINO,65.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO


In [ ]:
df_2025['ID'] = df_2025.index

df_2025['ID'] = df_2025['ID'].astype(int)

df_2025.head()

,ID,DATA,HORA_MINUTO,MUNICIPIO,BAIRRO,ENDERECO,ORIGEM_CHAMADO,TIPO,SUBTIPO,SEXO,IDADE,MOTIVO_FINALIZACAO,MOTIVO_DESFECHO
0,0,2025-01-01,00:00:45,RECIFE,BOA VISTA,AV DA BOA VISTA,RESIDENCIAL,DROGAS,ALCOOLISMO,MASCULINO,25.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
1,1,2025-01-01T00:00:00,00:08:03,POMBOS,ALTO DO FRADE,R SARDINHA,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO ENVOLVENDO MOTO,MASCULINO,22.0,NaN,1. OCORRÊNCIA CONCLUÍDA COM ÊXITO
2,2,2025-01-01T00:00:00,00:08:35,PAULISTA,NOSSA SENHORA DO O,AV CLAUDIO JOSE GUEIROS LEIT NOSSA.SENHORA.DO.O,RESIDENCIAL,CAUSAS EXTERNAS,QUEDA DA PROPRIA ALTURA,FEMININO,45.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
3,3,2025-01-01T00:00:00,00:17:44,JABOATAO DOS GUARARAPES,MARCOS FREIRE,RUA DOMINGO FERNANDES N,RESIDENCIAL,DROGAS,INTOXICACAO EXOGENA,FEMININO,46.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO
4,4,2025-01-01T00:00:00,00:19:24,IGARASSU,CRUZ DE REBOUCAS,R JOCA RODRIGUES,VIA PÚBLICA,GASTROINTESTINAL,DOR ABDOMINAL,MASCULINO,65.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO


In [5]:
df_2025.set_index('ID', inplace=True)

df_2025.head()

,DATA,HORA_MINUTO,MUNICIPIO,BAIRRO,ENDERECO,ORIGEM_CHAMADO,TIPO,SUBTIPO,SEXO,IDADE,MOTIVO_FINALIZACAO,MOTIVO_DESFECHO
ID,,,,,,,,,,,,
0,2025-01-01,00:00:45,RECIFE,BOA VISTA,AV DA BOA VISTA,RESIDENCIAL,DROGAS,ALCOOLISMO,MASCULINO,25.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
1,2025-01-01T00:00:00,00:08:03,POMBOS,ALTO DO FRADE,R SARDINHA,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO ENVOLVENDO MOTO,MASCULINO,22.0,NaN,1. OCORRÊNCIA CONCLUÍDA COM ÊXITO
2,2025-01-01T00:00:00,00:08:35,PAULISTA,NOSSA SENHORA DO O,AV CLAUDIO JOSE GUEIROS LEIT NOSSA.SENHORA.DO.O,RESIDENCIAL,CAUSAS EXTERNAS,QUEDA DA PROPRIA ALTURA,FEMININO,45.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
3,2025-01-01T00:00:00,00:17:44,JABOATAO DOS GUARARAPES,MARCOS FREIRE,RUA DOMINGO FERNANDES N,RESIDENCIAL,DROGAS,INTOXICACAO EXOGENA,FEMININO,46.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO
4,2025-01-01T00:00:00,00:19:24,IGARASSU,CRUZ DE REBOUCAS,R JOCA RODRIGUES,VIA PÚBLICA,GASTROINTESTINAL,DOR ABDOMINAL,MASCULINO,65.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO


In [6]:
df_2025.info()

<class 'pandas.core.frame.DataFrame'>
Index: 127304 entries, 0 to 127303
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   DATA                127304 non-null  object 
 1   HORA_MINUTO         127304 non-null  object 
 2   MUNICIPIO           127303 non-null  object 
 3   BAIRRO              127303 non-null  object 
 4   ENDERECO            119574 non-null  object 
 5   ORIGEM_CHAMADO      127300 non-null  object 
 6   TIPO                127301 non-null  object 
 7   SUBTIPO             124715 non-null  object 
 8   SEXO                125177 non-null  object 
 9   IDADE               110457 non-null  float64
 10  MOTIVO_FINALIZACAO  40854 non-null   object 
 11  MOTIVO_DESFECHO     127304 non-null  object 
dtypes: float64(1), object(11)
memory usage: 12.6+ MB


In [7]:
df_2025['IDADE'] = pd.to_numeric(df_2025['IDADE'], errors='coerce')

df_2025['DATA'] = pd.to_datetime(df_2025['DATA'], errors='coerce').dt.date

temp_hora = pd.to_datetime(df_2025['HORA_MINUTO'], errors='coerce')

df_2025['HORA_MINUTO'] = temp_hora.dt.time

df_2025.info()

/tmp/ipykernel_21124/18967384.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp_hora = pd.to_datetime(df_2025['HORA_MINUTO'], errors='coerce')


<class 'pandas.core.frame.DataFrame'>
Index: 127304 entries, 0 to 127303
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   DATA                1 non-null       object 
 1   HORA_MINUTO         127304 non-null  object 
 2   MUNICIPIO           127303 non-null  object 
 3   BAIRRO              127303 non-null  object 
 4   ENDERECO            119574 non-null  object 
 5   ORIGEM_CHAMADO      127300 non-null  object 
 6   TIPO                127301 non-null  object 
 7   SUBTIPO             124715 non-null  object 
 8   SEXO                125177 non-null  object 
 9   IDADE               110457 non-null  float64
 10  MOTIVO_FINALIZACAO  40854 non-null   object 
 11  MOTIVO_DESFECHO     127304 non-null  object 
dtypes: float64(1), object(11)
memory usage: 12.6+ MB


In [9]:
colunas_nomes = [
    'ID', 'DATA', 'HORA_MINUTO', 'MUNICIPIO', 'BAIRRO', 
    'ENDERECO', 'ORIGEM_CHAMADO', 'TIPO', 'SUBTIPO', 
    'SEXO', 'IDADE', 'MOTIVO_FINALIZACAO', 'MOTIVO_DESFECHO'
]
df_2025 = pd.read_csv('/home/pedro/Projetos/samu -  banco de dados/SAMU-Equipe-4/dados/Dados samu 2025.csv', header=None, names=colunas_nomes)

df_2025['ID'] = df_2025.index
df_2025.set_index('ID', inplace=True)


df_2025['DATA'] = df_2025['DATA'].astype(str).str.split('T').str[0]

df_2025['DATA'] = pd.to_datetime(df_2025['DATA'], errors='coerce').dt.date

df_2025['HORA_MINUTO'] = df_2025['HORA_MINUTO'].astype(str).str.strip().str[:8]

df_2025['HORA_MINUTO'] = pd.to_datetime(df_2025['HORA_MINUTO'], format='%H:%M:%S', errors='coerce').dt.time

df_2025['IDADE'] = pd.to_numeric(df_2025['IDADE'], errors='coerce')


print(df_2025[['DATA', 'HORA_MINUTO', 'IDADE']].info())
df_2025.head()

/tmp/ipykernel_21124/969901745.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2025 = pd.read_csv('/home/pedro/Projetos/samu -  banco de dados/SAMU-Equipe-4/dados/Dados samu 2025.csv', header=None, names=colunas_nomes)


<class 'pandas.core.frame.DataFrame'>
Index: 127304 entries, 0 to 127303
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   DATA         127304 non-null  object 
 1   HORA_MINUTO  127304 non-null  object 
 2   IDADE        110457 non-null  float64
dtypes: float64(1), object(2)
memory usage: 3.9+ MB
None


,DATA,HORA_MINUTO,MUNICIPIO,BAIRRO,ENDERECO,ORIGEM_CHAMADO,TIPO,SUBTIPO,SEXO,IDADE,MOTIVO_FINALIZACAO,MOTIVO_DESFECHO
ID,,,,,,,,,,,,
0,2025-01-01,00:00:45,RECIFE,BOA VISTA,AV DA BOA VISTA,RESIDENCIAL,DROGAS,ALCOOLISMO,MASCULINO,25.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
1,2025-01-01,00:08:03,POMBOS,ALTO DO FRADE,R SARDINHA,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO ENVOLVENDO MOTO,MASCULINO,22.0,NaN,1. OCORRÊNCIA CONCLUÍDA COM ÊXITO
2,2025-01-01,00:08:35,PAULISTA,NOSSA SENHORA DO O,AV CLAUDIO JOSE GUEIROS LEIT NOSSA.SENHORA.DO.O,RESIDENCIAL,CAUSAS EXTERNAS,QUEDA DA PROPRIA ALTURA,FEMININO,45.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
3,2025-01-01,00:17:44,JABOATAO DOS GUARARAPES,MARCOS FREIRE,RUA DOMINGO FERNANDES N,RESIDENCIAL,DROGAS,INTOXICACAO EXOGENA,FEMININO,46.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO
4,2025-01-01,00:19:24,IGARASSU,CRUZ DE REBOUCAS,R JOCA RODRIGUES,VIA PÚBLICA,GASTROINTESTINAL,DOR ABDOMINAL,MASCULINO,65.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO


In [10]:
print("Tipo real na coluna DATA:", type(df_2025['DATA'].iloc[0]))

print("Tipo real na coluna HORA:", type(df_2025['HORA_MINUTO'].iloc[0]))

Tipo real na coluna DATA: <class 'datetime.date'>
Tipo real na coluna HORA: <class 'datetime.time'>


In [11]:
mediana_idade = df_2025['IDADE'].median()

df_2025['IDADE'].fillna(mediana_idade, inplace=True)

qtd_nulos = df_2025['IDADE'].isnull().sum()

print("Quantidade de valores nulos na coluna IDADE após preenchimento:", qtd_nulos)

Quantidade de valores nulos na coluna IDADE após preenchimento: 0


/tmp/ipykernel_21124/3802179513.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_2025['IDADE'].fillna(mediana_idade, inplace=True)


In [12]:
df_2025['MOTIVO_FINALIZACAO'] = df_2025['MOTIVO_FINALIZACAO'].fillna('SEM FINALIZAÇÃO')

print("Quantidade de nulos após tratamento:", df_2025['MOTIVO_FINALIZACAO'].isnull().sum())

df_2025[['MOTIVO_FINALIZACAO']].info()

Quantidade de nulos após tratamento: 0
<class 'pandas.core.frame.DataFrame'>
Index: 127304 entries, 0 to 127303
Data columns (total 1 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   MOTIVO_FINALIZACAO  127304 non-null  object
dtypes: object(1)
memory usage: 1.9+ MB


In [14]:
df_2025['ENDERECO'] = df_2025['ENDERECO'].fillna('NÃO INFORMADO')

df_2025['ORIGEM_CHAMADO'] = df_2025['ORIGEM_CHAMADO'].fillna('NÃO INFORMADO')

print(df_2025[['ENDERECO', 'ORIGEM_CHAMADO']].isnull().sum())

df_2025.head()

ENDERECO          0
ORIGEM_CHAMADO    0
dtype: int64


,DATA,HORA_MINUTO,MUNICIPIO,BAIRRO,ENDERECO,ORIGEM_CHAMADO,TIPO,SUBTIPO,SEXO,IDADE,MOTIVO_FINALIZACAO,MOTIVO_DESFECHO
ID,,,,,,,,,,,,
0,2025-01-01,00:00:45,RECIFE,BOA VISTA,AV DA BOA VISTA,RESIDENCIAL,DROGAS,ALCOOLISMO,MASCULINO,25.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
1,2025-01-01,00:08:03,POMBOS,ALTO DO FRADE,R SARDINHA,VIA PÚBLICA,CAUSAS EXTERNAS,ACIDENTE DE TRANSITO ENVOLVENDO MOTO,MASCULINO,22.0,SEM FINALIZAÇÃO,1. OCORRÊNCIA CONCLUÍDA COM ÊXITO
2,2025-01-01,00:08:35,PAULISTA,NOSSA SENHORA DO O,AV CLAUDIO JOSE GUEIROS LEIT NOSSA.SENHORA.DO.O,RESIDENCIAL,CAUSAS EXTERNAS,QUEDA DA PROPRIA ALTURA,FEMININO,45.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
3,2025-01-01,00:17:44,JABOATAO DOS GUARARAPES,MARCOS FREIRE,RUA DOMINGO FERNANDES N,RESIDENCIAL,DROGAS,INTOXICACAO EXOGENA,FEMININO,46.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO
4,2025-01-01,00:19:24,IGARASSU,CRUZ DE REBOUCAS,R JOCA RODRIGUES,VIA PÚBLICA,GASTROINTESTINAL,DOR ABDOMINAL,MASCULINO,65.0,REMOVIDO ANTES DO ATENDIMENTO POR PARTICULARES,SEM DESFECHO
